<a href="https://colab.research.google.com/github/PardhaSaradhi74/Python/blob/master/Spam_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Importing Libraries
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

In [5]:
# Reading the Dataset
data = pd.read_csv('spam.csv', encoding="latin-1")

# Keeping only the neccessary columns
data = data[['v2','v1']]
data

,v2,v1
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


In [6]:
# Preprocessing the data 
data['v2'] = data['v2'].apply(lambda x: x.lower())
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
data

,v2,v1
0,go until jurong point crazy available only in ...,ham
1,ok lar joking wif u oni,ham
2,free entry in 2 a wkly comp to win fa cup fina...,spam
3,u dun say so early hor u c already then say,ham
4,nah i dont think he goes to usf he lives aroun...,ham
...,...,...
5567,this is the 2nd time we have tried 2 contact u...,spam
5568,will _ b going to esplanade fr home,ham
5569,pity was in mood for that soany other suggest...,ham
5570,the guy did some bitching but i acted like id ...,ham


In [7]:
# Tokenization,converting the data to sequenece of words
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)
X = pad_sequences(X)
X

array([[   0,    0,    0, ...,   67,   58,  137],
       [   0,    0,    0, ...,  443,    6, 1823],
       [   0,    0,    0, ...,  459,   79,  382],
       ...,
       [   0,    0,    0, ...,   12,   19,  231],
       [   0,    0,    0, ...,  198,   12,   50],
       [   0,    0,    0, ...,    1,   41,  258]], dtype=int32)

In [8]:
# Model Configuration, Embedding layer dimension, LSTM Out value
embed_dim = 128
lstm_out = 196

In [9]:
# Function to create the Model
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [10]:
# Applying label encoding on Target column
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [11]:
# Model Creation and Evalutaion
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 35s - loss: 0.1643 - accuracy: 0.9427


In [12]:
# Score and Accuracy Values
print(score)
print(acc)
print(model.metrics_names)

0.06470760104336075
0.9831430315971375
['loss', 'accuracy']
